
요청

헤더 -> session에 헤더에 추가해줘야함

Authorization: KakaoAK ${REST_API_KEY}

쿼리 파라미터

-> url에 사용


# 카카오 OpenAPI 연동 (이미지 검색)

## #01. 준비과정

### [1] 패키지 참조


In [11]:
import requests
import json
import datetime as dt
import concurrent.futures as futures
import os
from dotenv import load_dotenv

### [2] 접속할 데이터의 URL

In [12]:

urlFmt = 'https://dapi.kakao.com/v2/search/image?query={query}&page={page}&size={size}'



### [3] 요청 변수

In [13]:
query = '파이썬'
page = 1
size = 50

load_dotenv(dotenv_path = '../../../dot_env/.env')

kakao_key = os.environ.get("kakao_key")
key = kakao_key

## #02. 데이터 요청하기
### [1] 세션 생성

In [14]:
session = requests.Session()

session.headers.update({
    "Referer": "",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Authorization": f"KakaoAK {key}"
})

### [2]데이터 가져오기

마지막 페이지임을 의미하는 'meta > is_end' 라는 값이 'True'로 반환될 때 까지 'page'를 1씩 증가시키면서 반복 수행

In [15]:
page = 1
isEnd = False

# 수집된 데이터를 모아 놓을 빈 리스트
mydata = []

while not isEnd:
    url = urlFmt.format(query=query, page = page, size = size)

    try:
        r = session.get(url)
        
        if r.status_code != 200:
            msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
            raise Exception(msg)
    except Exception as e:
        print("접속에 실패했습니다.")
        print(e)

        # 에러 발생시 나머지 페이지 중단
        break

        # 에러 발생시 다음 페이지 시도
        # continue
    
    
    r.encoding = "utf-8"
    mydict = json.loads(r.text)

    mydata += mydict['documents']
    print(f'{page} 페이지로부터 {len(mydict['documents'])}건의 데이터를 수집했습니다.')

    page += 1 # 페이지 증가
    isEnd = mydict['meta']['is_end'] # 마지막 페이지인지 확인 // 연동하는 API에 따라 처리방법이 다름

print(mydata)
print('fin :)')

1 페이지로부터 50건의 데이터를 수집했습니다.
2 페이지로부터 50건의 데이터를 수집했습니다.
3 페이지로부터 50건의 데이터를 수집했습니다.
4 페이지로부터 50건의 데이터를 수집했습니다.
5 페이지로부터 50건의 데이터를 수집했습니다.
6 페이지로부터 50건의 데이터를 수집했습니다.
7 페이지로부터 50건의 데이터를 수집했습니다.
8 페이지로부터 50건의 데이터를 수집했습니다.
9 페이지로부터 50건의 데이터를 수집했습니다.
10 페이지로부터 50건의 데이터를 수집했습니다.
11 페이지로부터 50건의 데이터를 수집했습니다.
12 페이지로부터 50건의 데이터를 수집했습니다.
13 페이지로부터 50건의 데이터를 수집했습니다.
14 페이지로부터 50건의 데이터를 수집했습니다.
15 페이지로부터 50건의 데이터를 수집했습니다.
16 페이지로부터 50건의 데이터를 수집했습니다.
17 페이지로부터 50건의 데이터를 수집했습니다.
18 페이지로부터 50건의 데이터를 수집했습니다.
19 페이지로부터 50건의 데이터를 수집했습니다.
20 페이지로부터 50건의 데이터를 수집했습니다.
21 페이지로부터 50건의 데이터를 수집했습니다.
22 페이지로부터 50건의 데이터를 수집했습니다.
23 페이지로부터 50건의 데이터를 수집했습니다.
24 페이지로부터 50건의 데이터를 수집했습니다.
25 페이지로부터 50건의 데이터를 수집했습니다.
26 페이지로부터 50건의 데이터를 수집했습니다.
27 페이지로부터 50건의 데이터를 수집했습니다.
28 페이지로부터 50건의 데이터를 수집했습니다.
29 페이지로부터 50건의 데이터를 수집했습니다.
30 페이지로부터 50건의 데이터를 수집했습니다.
31 페이지로부터 50건의 데이터를 수집했습니다.
32 페이지로부터 50건의 데이터를 수집했습니다.
33 페이지로부터 50건의 데이터를 수집했습니다.
34 페이지로부터 50건의 데이터를 수집했습니다.
35 페이지로부터 50건의 데이터를 수집했습니다.
36 페이지로부터 50건의 데이터를 수집했습니다.
3

## #03. 데이터 활용

### [1] 수집 결과를 CSV로 저장하기



In [16]:
with open(f'{query}_이미지_검색결과.csv','w',encoding='utf-8') as f:
    f.write('컬렉션, 출처, 가로, 세로, 문서URL, 미리보기URL, \n')
    for i, v in enumerate(mydata):
        collection, thumbnail_url, image_url, width, height, display_sitename, doc_url, datetime = v.values()
        
        f.write(f"{collection}, {display_sitename},  {width}, {height}, {doc_url}, {thumbnail_url}\n")

print("fin :)")

    

fin :)


### [2] 이미지 다운로드
#### (1) 전체 데이터 수 확인

In [17]:
print(f'다운로드 받을 이미지 수 :{len(mydata)}')

다운로드 받을 이미지 수 :2038


#### (2) 다운로드 받을 폴더 생성

In [18]:
downloadDir = f'{query}_{dt.datetime.now().strftime('%y%m%d_%H%M%S')}'

if not os.path.exists(downloadDir):
    os.mkdir(downloadDir) # 다운받을때 마다 폴더생성

#### (3) 이미지 다운로드 함수 구현

동기 처리를 위해서는 반드시 정의해야 하는 것은 아니지만 비동기처리를 염두한다면 단위 기능을 함수로 묶어야 한다.

In [19]:
def kakaoImageDownload(session, docUrl, imageUrl, savePath):
    # 이미지 입장에서는 자신이 포함된 페이지가 이전 접속 페이지 이므로 referer 설정을 한다.

    session.headers.update({'Referer' : docUrl})
    try: 
        r = session.get(imageUrl, stream=True)
        if r.status_code != 200:
            msg = f'[{r.status_code} Error] {r.reason} 에러가 발생함'
            raise Exception(msg)
    except Exception as e :
        print('접속에 실패했습니다.', e, sep = '\n')
        
    # 인코딩 형식 지정
    r.encoding = "utf-8"

    # 수신 결과를 파일로 저장
    with open(savePath, "wb") as f:
        f.write(r.raw.read())


#### (4) 동기식 호출
작업을 하나씩 순차적으로 수행한다.

수행시간은 약 1분 15초

In [20]:
for i, v in enumerate(mydata):
    kakaoImageDownload(session, v['doc_url'], v['image_url'], "%s/%05d.png" % (downloadDir, i))

접속에 실패했습니다.
[404 Error] Not Found 에러가 발생함
접속에 실패했습니다.
[404 Error] Not Found 에러가 발생함
접속에 실패했습니다.
[404 Error] Not Found 에러가 발생함
접속에 실패했습니다.
[404 Error] Not Found 에러가 발생함
접속에 실패했습니다.
[403 Error] Access Denied 에러가 발생함
접속에 실패했습니다.
[404 Error] Not Found 에러가 발생함
접속에 실패했습니다.
[404 Error] Not Found 에러가 발생함
접속에 실패했습니다.
[403 Error] Access Denied 에러가 발생함
접속에 실패했습니다.
[403 Error] Access Denied 에러가 발생함
접속에 실패했습니다.
[403 Error] Access Denied 에러가 발생함
접속에 실패했습니다.
[403 Error] Access Denied 에러가 발생함
접속에 실패했습니다.
[404 Error] Not Found 에러가 발생함
접속에 실패했습니다.
[403 Error] Access Denied 에러가 발생함


#### (5) 비동기식 호출

수행시간은 약 22초


In [21]:
downloadDir = f'{query}_{dt.datetime.now().strftime('%y%m%d_%H%M%S')}'

if not os.path.exists(downloadDir):
    os.mkdir(downloadDir) #  새 폴더 생성

In [23]:
with futures.ThreadPoolExecutor(max_workers=10) as executor:
    for i, v in enumerate(mydata):
        executor.submit(kakaoImageDownload,session, v['doc_url'], v['image_url'], "%s/%05d.png" % (downloadDir, i))
print('fin:)')

접속에 실패했습니다.
[404 Error] Not Found 에러가 발생함
접속에 실패했습니다.
[403 Error] Forbidden 에러가 발생함
접속에 실패했습니다.
[403 Error] Access Denied 에러가 발생함
접속에 실패했습니다.
[403 Error] Forbidden 에러가 발생함
접속에 실패했습니다.
[403 Error] Access Denied 에러가 발생함
접속에 실패했습니다.
[404 Error] Not Found 에러가 발생함
접속에 실패했습니다.
[404 Error] Not Found 에러가 발생함
접속에 실패했습니다.
[403 Error] Access Denied 에러가 발생함
접속에 실패했습니다.
[403 Error] Access Denied 에러가 발생함
접속에 실패했습니다.
[403 Error] Access Denied 에러가 발생함
접속에 실패했습니다.
[403 Error] Access Denied 에러가 발생함
접속에 실패했습니다.
[403 Error] Access Denied 에러가 발생함
접속에 실패했습니다.
[403 Error] Access Denied 에러가 발생함
접속에 실패했습니다.
[404 Error] Not Found 에러가 발생함
접속에 실패했습니다.
[403 Error] Access Denied 에러가 발생함
접속에 실패했습니다.
[403 Error] Access Denied 에러가 발생함
접속에 실패했습니다.
[403 Error] Access Denied 에러가 발생함
접속에 실패했습니다.
[403 Error] Access Denied 에러가 발생함
접속에 실패했습니다.
[403 Error] Access Denied 에러가 발생함
접속에 실패했습니다.
[403 Error] Access Denied 에러가 발생함
접속에 실패했습니다.
[404 Error] Not Found 에러가 발생함
접속에 실패했습니다.
[404 Error] Not Found 에러가 발생함
접속에 실패했습니다.
[403 Err